In [35]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 30.3 MB/s eta 0:00:00


# 감성분석 실습
- 긍정/부정 리뷰에 영향을 많이 주는 단어들이 무엇인지 인사이트를 얻어보자

In [62]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt # konlpy 형태소 단위로 분리
from tqdm import tqdm
# 분류평가지표 확인
from sklearn.metrics import classification_report

In [ ]:
#컬럼 안의 글자 모두 확인
pd.set_option('display.max_colwidth', 100)

In [7]:
%cd '/content/drive/MyDrive/Colab Notebooks/Ai_Deep_Learning/'

/content/drive/MyDrive/Colab Notebooks/Ai_Deep_Learning


In [43]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Ai_Deep_Learning/data/steam.txt', delimiter='\t', header=None)


In [45]:
train.columns=['평점','댓글']

,평점,댓글
0,0,노래가 너무 적음
1,0,"돌겠네 진짜. 황숙아, 어크 공장 그만 돌려라. 죽는다."
2,1,막노동 체험판 막노동 하는사람인데 장비를 내가 사야돼 뭐지
3,1,차악!차악!!차악!!! 정말 이래서 왕국을 되찾을 수 있는거야??
4,1,시간 때우기에 좋음.. 도전과제는 50시간이면 다 깰 수 있어요
...,...,...
99995,0,한글화해주면 10개산다
99996,0,개쌉노잼 ㅋㅋ
99997,0,노잼이네요... 30분하고 지웠어요...
99998,1,야생을 사랑하는 사람들을 위한 짧지만 여운이 남는 이야기. 영어는 그리 어렵지 않습니다.


In [29]:
# X_train, X_test, y_train, y_test = train_test_split(train, train['label'], test_size=0.2, random_state=42)

In [47]:
def labeling(row):
  if row['평점'] > 0:
    return 1
  else :
    return 0

In [48]:
train['label'] =train.apply(labeling, axis=1)

,평점,댓글,label
0,0,노래가 너무 적음,0
1,0,"돌겠네 진짜. 황숙아, 어크 공장 그만 돌려라. 죽는다.",0
2,1,막노동 체험판 막노동 하는사람인데 장비를 내가 사야돼 뭐지,1
3,1,차악!차악!!차악!!! 정말 이래서 왕국을 되찾을 수 있는거야??,1
4,1,시간 때우기에 좋음.. 도전과제는 50시간이면 다 깰 수 있어요,1
...,...,...,...
99995,0,한글화해주면 10개산다,0
99996,0,개쌉노잼 ㅋㅋ,0
99997,0,노잼이네요... 30분하고 지웠어요...,0
99998,1,야생을 사랑하는 사람들을 위한 짧지만 여운이 남는 이야기. 영어는 그리 어렵지 않습니다.,1


/usr/local/lib/python3.10/dist-packages/google/colab/_quickchart_lib.py:75: UserWarning: Glyph 54217 (\N{HANGUL SYLLABLE PYEONG}) missing from current font.
  plt.tight_layout()
/usr/local/lib/python3.10/dist-packages/google/colab/_quickchart_lib.py:75: UserWarning: Glyph 51216 (\N{HANGUL SYLLABLE JEOM}) missing from current font.
  plt.tight_layout()
/usr/local/lib/python3.10/dist-packages/google/colab/_quickchart_lib.py:26: UserWarning: Glyph 54217 (\N{HANGUL SYLLABLE PYEONG}) missing from current font.
  plt.savefig(
/usr/local/lib/python3.10/dist-packages/google/colab/_quickchart_lib.py:26: UserWarning: Glyph 51216 (\N{HANGUL SYLLABLE JEOM}) missing from current font.
  plt.savefig(
/usr/local/lib/python3.10/dist-packages/google/colab/_quickchart_lib.py:146: UserWarning: Glyph 54217 (\N{HANGUL SYLLABLE PYEONG}) missing from current font.
  plt.tight_layout()
/usr/local/lib/python3.10/dist-packages/google/colab/_quickchart_lib.py:146: UserWarning: Glyph 51216 (\N{HANGUL SYLLABLE JEO

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(train, *['평점'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(train, *['label'], **{})
chart

import numpy as np
from google.colab import autoviz

def scatter_plot(df, x_colname, y_colname, figscale=1, alpha=.8):
  from matplotlib import pyplot as plt
  plt.figure(figsize=(6 * figscale, 6 * figscale))
  df.plot(kind='scatter', x=x_colname, y=y_colname, s=(32 * figscale), alpha=alpha)
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = scatter_plot(train, *['평점', 'label'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(train, *['평점'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(train, *['label'], **{})
chart

In [51]:
X = train['댓글']
y = train['label']

In [54]:
okt = Okt() # 형태소 분석기 생성
train_morphs = [] # 형태소 단위로 쪼개진 리뷰를 담을 리스트 생성
for doc in tqdm(X): # 약 100000개 리뷰 반복
  morphs_rs = okt.morphs(doc) # 형태소 단위로 분리
  train_morphs.append(" ".join(morphs_rs))

100%|██████████| 100000/100000 [08:08<00:00, 204.63it/s]


In [55]:
steam_tfidf = TfidfVectorizer(stop_words=['은','는','이','가'],
                             min_df = 3, max_df = 1000) # tfidf 객체 생성
steam_tfidf.fit(train_morphs) # 토큰화 및 단어사전 구축
print('단어길이',len(steam_tfidf.vocabulary_)) # 단어사전의 길이 확인
X_train = steam_tfidf.transform(train_morphs) # 수치화
print(X_train.shape)

단어길이 26226
(100000, 26226)


In [56]:
# 선형분류 모델활용 학습 및 평가 -> 교차검증 활용
steam_logi = LogisticRegression()
steam_logi


LogisticRegression()

In [58]:
score = cross_val_score(steam_logi, X_train, y, cv=3 )
score

array([0.75370493, 0.75360754, 0.75432754])

In [59]:
steam_logi.fit(X_train,y )

LogisticRegression()

In [61]:
steam_logi.score(X_train,y)

0.81777

In [63]:
pre_val = steam_logi.predict(X_train)
print(classification_report(y, pre_val))

              precision    recall  f1-score   support

           0       0.83      0.80      0.82     50004
           1       0.81      0.83      0.82     49996

    accuracy                           0.82    100000
   macro avg       0.82      0.82      0.82    100000
weighted avg       0.82      0.82      0.82    100000

